In [ ]:
#掛接雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')
import pandas as pd
import wordcloud 
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from ipywidgets import interact
import seaborn as sns
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet, stopwords as sw
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

train_data = pd.read_table('train.txt', sep=";" , header = None).rename(columns = {0: 'sentence', 1: 'label'})
valid_data = pd.read_table('val.txt', sep=";" , header = None).rename(columns = {0: 'sentence', 1: 'label'})

train_valid_data = pd.concat([train_data, valid_data], ignore_index=True)
train_valid_data

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
17995,im having ssa examination tomorrow in the morn...,sadness
17996,i constantly worry about their fight against n...,joy
17997,i feel its important to share this info for th...,joy
17998,i truly feel that if you are passionate enough...,joy


In [ ]:
def preprocessing(dataframe,str_):
  #將 sentence 每一個單字拆開 
  sentence = dataframe["sentence"].tolist()
  nest_list = []
  all_word = []
  i = 0
  max_words = 0
  while i < len(sentence):
    all_word.extend(sentence[i].split(" "))
    nest_list.append(sentence[i].split(" "))
    i = i + 1

  ## 停用詞整合
  stopwords = list(wordcloud.STOPWORDS)
  nltk_stopwords = list(sw.words('english')) #stopwords.words('english')
  i = 0
  while i < len(nltk_stopwords):
    if nltk_stopwords[i] not in stopwords:
      stopwords.append(nltk_stopwords[i])
    i = i + 1

  i = 0
  while i < len(stopwords):
    if "'" in stopwords[i] and "'t" not in stopwords[i]:
      stopwords[i] = stopwords[i].replace("'","")
    i = i + 1 

  ## lemmatization 字型還原
  # 獲取單詞的詞性  “n” for nouns, “v” for verbs, “a” for adjectives, “r” for adverbs
  def get_wordnet_pos(tag):
    if tag.startswith('J'):
      return "a"
    elif tag.startswith('V'):
      return "v"
    elif tag.startswith('N'):
      return "n"
    elif tag.startswith('R'):
      return "r"
    else:
      return "n"

  def lemma(tokens):
    tagged_sent = pos_tag(tokens)     # 獲取單詞詞性
    wnl = WordNetLemmatizer()
    lemmas_sent = []
    for tag in tagged_sent:
      if get_wordnet_pos(tag[1]) != "n":  #不放入名詞
        lemmas_sent.append(wnl.lemmatize(tag[0], pos = get_wordnet_pos(tag[1]))) # 詞形還原
    return lemmas_sent

  lemma_nest_list = []
  lemma_all_word = []
  i = 0
  while i < len(nest_list):
    lemma_nest_list.append(lemma(nest_list[i]))
    lemma_all_word.extend(lemma(nest_list[i]))
    i = i + 1

  no_sw_lemma_all_word = []
  no_sw_lemma_nest_list = lemma_nest_list
  no_sw_lemma_max_words = 0
  num_word = []
  zero_word_index = []

  i = 0
  while i < len(no_sw_lemma_nest_list):
    j = len(no_sw_lemma_nest_list[i]) - 1
    while j >= 0:
      if no_sw_lemma_nest_list[i][j] in stopwords:
        no_sw_lemma_nest_list[i].remove(no_sw_lemma_nest_list[i][j])
        j = j - 1
      else:
        j = j - 1
    if no_sw_lemma_max_words < len(no_sw_lemma_nest_list[i]):
      no_sw_lemma_max_words = len(no_sw_lemma_nest_list[i])
    
    if len(no_sw_lemma_nest_list[i]) == 0:
      zero_word_index.append(i)
    num_word.append(len(no_sw_lemma_nest_list[i]))
    no_sw_lemma_all_word.extend(no_sw_lemma_nest_list[i])
    i = i + 1
  
  new_col_name = []
  i = 0
  while i < no_sw_lemma_max_words:
    new_col_name.append("word" + str(i + 1))
    i = i + 1

  #df = pd.DataFrame(a, columns=['D0', 'D1'])
  seg_words = pd.DataFrame(no_sw_lemma_nest_list, columns = new_col_name)
  #seg_words.loc[[237,6322,9753,11400]][:]

  no_sw_lemma_all_word_dict = Counter(no_sw_lemma_all_word)
  no_sw_lemma_unique_word = list(no_sw_lemma_all_word_dict.keys())
  

  def get_dummy(list_):
    list_counter = Counter(list_) #list = after_tf_idf_nest_list[j]
    list_counter = list(list_counter.items())
    index = []
    i = 0
    while i < len(list_counter): #list.index(element)
      if list_counter[i][0] in no_sw_lemma_unique_word:
        index.append(no_sw_lemma_unique_word.index(list_counter[i][0]))
      i = i + 1 
    count_list = [0] * len(no_sw_lemma_unique_word)
    i = 0
    while i < len(index):
      count_list[index[i]] = list_counter[i][1] 
      i = i + 1
    return count_list
  
  count_nest_list = []
  j = 0
  while j < len(no_sw_lemma_nest_list):
    count_nest_list.append(get_dummy(no_sw_lemma_nest_list[j]))
    j = j + 1
  dummy = pd.DataFrame(count_nest_list, columns = no_sw_lemma_unique_word)
  dummy.to_csv('/content/drive/My Drive/Colab Notebooks/' + str_ + '_dummy.csv')
  return dummy

In [ ]:
preprocessing(train_valid_data, 'train_valid')

,didnt,humiliate,go,feel,hopeless,damned,hopeful,care,awake,grab,post,wrong,nostalgic,know,still,little,burdened,lately,wasnt,sure,take,recommended,fall,asleep,faster,funny,confuse,jade,old,perform,make,huge,romantic,see,run,divine,expect,spiritual,think,easy,...,unruly,leather,banish,oneself,nyc,farthest,lous,staffed,exceed,leanne,wll,excotick,yummyy,deisire,enjoin,functional,tested,sewing,magically,hometown,universal,sync,accessible,dave,helen,familiarly,regroup,lolll,blissful,healing,bellmont,parisian,moreover,mandate,mash,jrock,internalize,objectively,smarter,ssa
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17997,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17998,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
